In [1]:
import sys
import glob
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# general IC imports
from invisible_cities.core.configure         import configure
from invisible_cities.database               import load_db
from invisible_cities.core.system_of_units_c import units
adc, pes, mus = units.adc, units.pes, units.mus
NN = -999999

# IRENE
from invisible_cities.cities.components import deconv_pmt
from invisible_cities.cities.components import calibrate_pmts
from invisible_cities.cities.components import calibrate_sipms

from invisible_cities.cities.components import deconv_pmt
from invisible_cities.cities.components import calibrate_pmts
from invisible_cities.cities.components import calibrate_sipms
from invisible_cities.cities.components import zero_suppress_wfs

from invisible_cities.reco.peak_functions import split_in_peaks
from invisible_cities.reco.peak_functions import select_peaks
from invisible_cities.reco.peak_functions import select_wfs_above_time_integrated_thr
from invisible_cities.reco.peak_functions import pick_slice_and_rebin

from invisible_cities.types.ic_types import minmax

# PENTHESILEA
from invisible_cities.reco.peak_functions import rebin_times_and_waveforms

# ESMERALDA
from invisible_cities.reco.corrections_new import read_maps
from invisible_cities.reco.corrections_new import apply_all_correction
from invisible_cities.reco.corrections_new import norm_strategy

In [2]:
config = "config.conf"
config = configure(["script", config]).as_namespace

# config = configure(sys.argv).as_namespace

In [3]:
# cut params
s1emin = config.s1emin
s1wmin = config.s1wmin

nS1s = config.nS1s
nS2s = config.nS2s

pmt_ids = np.array( config.active_pmts )

#IRENE params
wf_dir     = config.wf_dir

run_number = config.run_number
run        = config.run
n_baseline = config.n_baseline
n_mau      = config.n_mau
thr_mau    = config.thr_mau
thr_csum_s1 = config.thr_csum_s1
thr_csum_s2 = config.thr_csum_s2
thr_sipm      = config.thr_sipm
thr_sipm_type = config.thr_sipm_type
s1_tmin         = config.s1_tmin
s1_tmax         = config.s1_tmax
s1_stride       = config.s1_stride
s1_lmin         = config.s1_lmin
s1_lmax         = config.s1_lmax
s1_rebin_stride = config.s1_rebin_stride
s2_tmin         = config.s2_tmin
s2_tmax         = config.s2_tmax
s2_stride       = config.s2_stride
s2_lmin         = config.s2_lmin
s2_lmax         = config.s2_lmax
s2_rebin_stride = config.s2_rebin_stride
thr_sipm_s2 = config.thr_csum_s2
detector_db = config.detector_db

#Penthesilea
qth_penth = config.qth_penth
rebin     = config.rebin

#Esmeralda
qth_esmer  = config.qth_esmer
map_file   = config.map_file
apply_temp = config.apply_temp

if thr_sipm_type.lower() == "common": 
    sipm_thr = thr_sipm

In [4]:
def signals_selected_splits(s1_indices, s2_indices,
                            s1_stride , s2_stride ,
                            s1_tmin, s1_tmax, s1_lmin, s1_lmax,
                            s2_tmin, s2_tmax, s2_lmin, s2_lmax):

    indices_split   = split_in_peaks(s1_indices, s1_stride)
    s1_selected_splits = select_peaks  (indices_split, 
                                        minmax(min = s1_tmin, max = s1_tmax), 
                                        minmax(min = s1_lmin, max = s1_lmax))

    indices_split   = split_in_peaks(s2_indices, s2_stride)
    s2_selected_splits = select_peaks  (indices_split, 
                                        minmax(min = s2_tmin, max = s2_tmax), 
                                        minmax(min = s2_lmin, max = s2_lmax))
    
    return s1_selected_splits, s2_selected_splits

In [5]:
def _1s1_1s2(pmt_ccwfs, s2_selected_splits, s1_selected_splits,
             s1emin   , s1wmin):
    ######## 1S1 1S2 CUT #########
    # 1S1 cut
    if len(s1_selected_splits)==0:
        return None
    # 1S2 cut
    if len(s2_selected_splits)>nS1s:
        return None
        
    # S1 energy and width cut
    s1es, s1ws = [], []
    for ss in s1_selected_splits:
        s1_pmt = np.sum( pmt_ccwfs[:, ss[0]: ss[-1]], axis=0)
        s1es.append( np.sum(s1_pmt)    )
        s1ws.append( (ss[-1]-ss[0])*25 )
    s1es, s1ws = np.array(s1es), np.array(s1ws)

    sel = (s1es>=s1emin) & (s1ws>=s1wmin)
    idxs = np.argwhere(sel).flatten()

    if len(idxs)==0:
        return None
    elif len(idxs)>1:
        return None
    else:
        idx = idxs[0]
        s1_pmt = np.sum( pmt_ccwfs[:, s1_selected_splits[idx][0]: s1_selected_splits[idx][-1]], axis=0)
        times  = np.arange(s1_selected_splits[idx][0], s1_selected_splits[idx][-1])*25

        S1_time = times[np.argmax(s1_pmt)]
        return S1_time

In [6]:
def create_penthesilea_hits(s2_pmts_penth, s2_sipms_penth,
                            sipms_xs     , sipms_ys      , sipm_ids,
                            times        , S1_time):
    ###### create penthesilea hits ########
    n_sipms = len(sipm_ids)
    X, Y = sipm_xs[sipm_ids], sipm_ys[sipm_ids]
    T = (times - S1_time)/1000

    E_per_slice = np.sum( s2_pmts_penth, axis=0)
    hits    = []
    nn_hits = []
    for t, e, q in zip(T, E_per_slice, s2_sipms_penth.T):
        if np.sum(q)==0:
            nn_hits.append( (0, 0, t, e, NN, -1) )
        else:
            E = e * q / np.sum(q)
            hits.append( (X, Y, np.full( n_sipms, t), E, q, np.full( n_sipms, -1) ) )
    hits = np.array( hits )
    hits = np.swapaxes(hits, axis1=1, axis2=2)
    hits = np.concatenate( hits )
    H = np.array(np.zeros(np.shape(hits)[0]), 
                 dtype=[("X", int)  , ("Y", int)  , ("Z", float), 
                        ("E", float), ("Q", float), ("Ec",float)])
    H["X"], H["Y"], H["Z"]  = hits[:, 0], hits[:, 1], hits[:, 2]
    H["E"], H["Q"], H["Ec"] = hits[:, 3], hits[:, 4], -1
    
    #### remove 0 charge hits and insert NN ####
    sel = ~(H["Q"]==0)
    H = np.insert( H[sel], 0, nn_hits)
    H = np.sort( H, order="Z")
    hits = H
    
    return hits

In [7]:
def esmeralda_charge_cut(hits, qth_esmer):
    #### Charge cut ####
    sel = (hits["Q"]>=qth_esmer)
    hits["Q"][~sel] = 0

    slides = np.unique( hits["Z"] )
    for slide in slides:
        sel = (hits["Z"]==slide)
        slide_hits = hits[sel]
        q = slide_hits["Q"]
        e = slide_hits["E"]
        if np.sum( q ) == 0:
            idxs = np.argwhere(sel).flatten()
            hits = np.delete(hits, idxs)
            hits = np.insert(hits, 0, (0, 0, slide, np.sum(e), NN, -1))
        else:
            hits["E"][sel] = np.sum( e ) * q / np.sum(q)
    sel = (hits["Q"]==0)
    hits = np.delete( hits, np.argwhere(sel))
    hits = np.sort(hits, order="Z")
    return hits

In [8]:
def join_NN_hits(hits):
    
    ###### join NN hits ######
    sel = (hits["Q"]==NN)
    nn_hits = hits[ sel]
    hits    = hits[~sel]
    slides = np.unique( hits["Z"] )
    for nn_hit in nn_hits:
        #select slide to append
        d = np.abs( slides - nn_hit["Z"] ) 
        slide = slides[ np.argmin( d ) ]
        slide_hits = hits[hits["Z"]==slide]
        #new energy 
        new_E = np.sum(slide_hits["E"]) + nn_hit["E"]
        q = hits[hits["Z"]==slide]["Q"]
        Q = np.sum( q )
        hits["E"][hits["Z"] == slide] = new_E * q / Q
    
    return hits

In [9]:
# FILE OUT
outdir = "./"

h5file = tb.open_file(outdir + "/" + f"_{run}_thrsipms2_{int(thr_sipm_s2)}_thrsipm_{int(thr_sipm)}.h5", 
                      mode="w", title="DE RWF data")

group = h5file.create_group("/", "Summary", "Summary")
h5file.create_earray(group, "Z" , tb.Float64Atom(), shape=(0, ))
h5file.create_earray(group, "DZ", tb.Float64Atom(), shape=(0, ))
h5file.create_earray(group, "E" , tb.Float64Atom(), shape=(0, ))
h5file.create_earray(group, "Q" , tb.Float64Atom(), shape=(0, ))
h5file.create_earray(group, "Ec", tb.Float64Atom(), shape=(0, ))

group  = h5file.create_group("/", f"Event_Info", "Info")

class Event_Info(tb.IsDescription):
    event = tb.Int32Col()
    time  = tb.UInt64Col()
    
Event_Info_table = h5file.create_table(group, "Event_Time", Event_Info, "Event_Time")
EI = Event_Info_table.row

In [10]:
files_in = glob.glob( wf_dir + "/*" )
files_in.sort()

In [11]:
datasipm = load_db.DataSiPM("new", run)
sipm_xs  = datasipm.X.values
sipm_ys  = datasipm.Y.values

maps = read_maps( map_file )
total_correction = apply_all_correction(maps, apply_temp=apply_temp,
                                        norm_strat=norm_strategy.kr)

In [12]:
import time
t0 = time.time()

_Z, _DZ, _E, _Q, _Ec = [], [], [], [], []

for file in files_in[0:2]:
    
    print( file )
    
    RWFs_file = tb.open_file( file )
    pmt_rwfs_all  = RWFs_file.root.RD.pmtrwf
    sipm_rwfs_all = RWFs_file.root.RD.sipmrwf
    time_stamps   = RWFs_file.root.Run.events.read()
    
    for event_time, pmt_rwfs, sipm_rwfs in zip(time_stamps, pmt_rwfs_all, sipm_rwfs_all):
        
        ################################
        ############ IRENE #############
        ################################
        
        #pmt processing
        rwf_to_cwf  = deconv_pmt    (detector_db, run_number, n_baseline)
        pmt_cwfs    = rwf_to_cwf    (pmt_rwfs)
        cwf_to_ccwf = calibrate_pmts(detector_db, run_number, n_mau, thr_mau)
        pmt_ccwfs, ccwfs_mau, cwf_sum, cwf_sum_mau  = cwf_to_ccwf(pmt_cwfs)
        #select pmt_ids wfs
        c = np.zeros(pmt_ccwfs.shape[0])
        c[pmt_ids] = 1
        pmt_ccwfs  = np.multiply( c, pmt_ccwfs.T ).T
        
        #sipm processing
        sipm_rwf_to_cal = calibrate_sipms(detector_db, run_number, sipm_thr)
        sipm_cwfs = sipm_rwf_to_cal(sipm_rwfs)
        
        
        #Find signals
        zero_suppress = zero_suppress_wfs(thr_csum_s1, thr_csum_s2)
        s1_indices, s2_indices = zero_suppress(cwf_sum, cwf_sum_mau)
    
        s1_selected_splits,\
        s2_selected_splits = signals_selected_splits(s1_indices, s2_indices,
                                                     s1_stride , s2_stride ,
                                                     s1_tmin, s1_tmax, s1_lmin, s1_lmax,
                                                     s2_tmin, s2_tmax, s2_lmin, s2_lmax)
        
        ######## 1S1 1S2 CUT ##########
        S1_time = _1s1_1s2(pmt_ccwfs, s2_selected_splits, s1_selected_splits,
                           s1emin   , s1wmin)
        if not S1_time: continue
        
        
        # Rebin S2_pmts
        times, rebinned_widths, s2_pmts = pick_slice_and_rebin(s2_selected_splits[0], 
                                                               np.arange(pmt_ccwfs.shape[1]) * 25 * units.ns, 
                                                               np.full  (pmt_ccwfs.shape[1],   25 * units.ns),
                                                               pmt_ccwfs, 
                                                               rebin_stride = s2_rebin_stride, 
                                                               pad_zeros    = True)
        #select and thr_sipm_s2
        s2_sipms = sipm_cwfs[:, s2_selected_splits[0][0] //40 : s2_selected_splits[0][-1]//40 + 1]
        sipm_ids, s2_sipms = select_wfs_above_time_integrated_thr(s2_sipms, thr_sipm_s2)
        
        ######## IRENE FINAL S2 WFS #######
        s2_pmts  = np.float32( s2_pmts )
        s2_sipms = np.float32( s2_sipms)
        times    = np.float32( times   )
        
        
        ################################
        ######## PENTHESILEA ###########
        ################################
        
        ########## Rebin ############
        _,     _, s2_sipms = rebin_times_and_waveforms(times, rebinned_widths, s2_sipms,
                                                       rebin_stride=rebin, slices=None)
        times, _, s2_pmts  = rebin_times_and_waveforms(times, rebinned_widths, s2_pmts,
                                                       rebin_stride=rebin, slices=None)
        ######### Charge cut #########
        s2_pmts_penth  = np.copy( s2_pmts )
        s2_sipms_penth = np.where(s2_sipms >= qth_penth, s2_sipms, 0)
        
        ###### create penthesilea hits ########
        hits = create_penthesilea_hits(s2_pmts_penth, s2_sipms_penth,
                                       sipm_xs      , sipm_ys       , sipm_ids,
                                       times        , S1_time)
    
        
        ################################
        ######### ESMERALDA ############
        ################################
        
        #### Charge cut ####
        hits = esmeralda_charge_cut(hits, qth_esmer)
        
        ###### join NN hits ######
        hits = join_NN_hits(hits)
            
        #### Corrections ######
        X, Y, Z = hits["X"], hits["Y"], hits["Z"]
        E, Q    = hits["E"], hits["Q"]
        T = np.full(len(hits), event_time[-1]/1000)
        correction_factor = total_correction(X, Y, Z, T)
        Ec = correction_factor * E
        hits["Ec"] = Ec
        hits["Z"]  = Z * maps.t_evol.dv.mean()
        
        
        ###########################
        ####### APPEND DATA #######
        ###########################
        # Event Info
        EI["event"] = event_time[0] 
        EI["time"]  = event_time[1]
        EI.append()
        
        ## Z, DZ, E, Q, Ec
        Z, E, Q, Ec = hits["Z"], hits["E"], hits["Q"], hits["Ec"]
        Ec[ np.isnan(Ec) ] = 0
        
        _Z .append( np.sum( Ec * Z) / np.sum(Ec) )
        _DZ.append( np.max(Z) - np.min(Z) )
        _E .append( np.sum(E)  )
        _Q .append( np.sum(Q)  )
        _Ec.append( np.sum(Ec) )
        
    # close RWF file
    RWFs_file.close()

h5file.root.Summary.Z .append( _Z  )
h5file.root.Summary.DZ.append( _DZ )
h5file.root.Summary.E .append( _E  )
h5file.root.Summary.Q .append( _Q  )
h5file.root.Summary.Ec.append( _Ec )

#write to disk
h5file.flush()
h5file.close()

print("Exec time", (time.time() - t0)/60)

/home/gdiaz/Verify_IRENENB_corrections/DATA/7430/rwf/run_7430_0000_trigger2_waveforms.h5
/home/gdiaz/Verify_IRENENB_corrections/DATA/7430/rwf/run_7430_0001_trigger2_waveforms.h5
Exec time 0.9057970523834229


In [14]:
h5file = tb.open_file(f"_{run}_thrsipms2_{int(thr_sipm_s2)}_thrsipm_{int(thr_sipm)}.h5")

In [15]:
events = h5file.root.Event_Info.Event_Time.read()["event"]

In [16]:
re  = np.random.choice( events )
idx = np.argwhere( events == re ).flatten()

In [17]:
h5file.root.Summary.Ec.read()[idx]

array([1.34305247])

In [18]:
h5file.root.Summary.E.read()[idx]

array([291122.01384711])

In [19]:
h5file.root.Summary.DZ.read()[idx]

array([43.33117034])

# CDSTs from production

In [20]:
datadir = f"/home/gdiaz/Verify_IRENENB_corrections/DATA/{run}/cdst/"

files = glob.glob( datadir + "/*")
files.sort()

h5file = tb.open_file( files[0] )

In [21]:
# event_time = h5file.root.Run.events.read()

#chits   = h5file.root.CHITS.lowTh.read()
chits_hTh   = h5file.root.CHITS.highTh.read()

#tracks  = h5file.root.Tracking.Tracks.read()
dst     = h5file.root.DST    .Events.read()
summary = h5file.root.Summary.Events.read()

In [22]:
chits = np.sort( chits_hTh[chits_hTh["event"]==re], order="Z")

In [23]:
np.sum( chits["E"] )

291122.01384711266

In [26]:
np.sum( chits["Ec"][~np.isnan( chits["Ec"] ) ] )

1.3430524687452845

In [25]:
np.max(chits["Z"]) - np.min( chits["Z"] )

43.33117033782332